In [1]:
#%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import math
#import w_ipa
import westpa.cli.tools.w_ipa as w_ipa
w = w_ipa.WIPI()
# At startup, it will load or run the analysis schemes specified in the configuration file (typically west.cfg)
w.main()
w.interface = 'matplotlib'


Welcome to w_ipa (WESTPA Interactive Python Analysis) v. 1.0B!
Run w.introduction for a more thorough introduction, or w.help to see a list of options.
Running analysis & loading files.
Reanalyzing file assign.h5 for scheme TEST.
Initializing...
Scanning for segment and point counts...
Preparing output...
Assigning to bins...
Reanalyzing file direct.h5 for scheme TEST.
Initializing...
Tracing trajectories...
Calculating Rate Evolution...
Calculating Conditional Flux Evolution...
Calculating Target Flux Evolution...
Calculating Rate Evolution...
Calculating Conditional Flux Evolution...
Calculating Target Flux Evolution...
Calculating Color Probability Evolution...
Calculating State Probability Evolution...
Calculating Color Probability Evolution...
Calculating State Probability Evolution...

Complete!
Setting iteration to iter 300.
Your current scheme, system and iteration are : TEST, /home/praveen/Documents/WE_HLDA/Ala/Ala_HLDA_WE/HLDA_WE_Simulations/Trial4/Red_Updated, 300


In [2]:
# Set the iteration to the second to last one.
w.iteration -= 1

# Access the current iteration dictionary, and return an interesting segment:
# the heaviest one in the current segment!
# Note that we can access either as attributes or dictionary keys!
# Attributes can be tab completed when running in console mode.
heaviest_segment = w.current.maxweight.seg_id
print('{} is the walker with the maximum weight!\n'.format(heaviest_segment))
# We can index the current iteration according to the seg_id...
print('Indexing according to a seg_id returns all properties on the segment.')
print('Children and parents have the same properties/keys.')
print(str(w.current[heaviest_segment]) + '\n')
# ... we can also see what its parents were like, or what any children are like, should they exist!
print('Where did it come from?  Does it have any descendents?')
print('{} is the parent!'.format(w.past[heaviest_segment].seg_id))
print('{} is/are the children!\n'.format(w.future[heaviest_segment].seg_id))

# We can see that the past/future objects are keyed to seg_ids in the CURRENT iteration, always.
# We can also see all available properties on the current iteration, and index THOSE according to a segment.
# As an example:
print('Can we index either way?')
print(str((w.current.pcoord[heaviest_segment] == w.current[heaviest_segment].pcoord)[0]) + '\n')

# w.current also contains the output from w_direct/w_reweight/w_assign for the current iteration.
# w.past contains the same output, from the prior iteration, keyed to the current seg_id
print('What properties exist in the current dictionary?')
print(w.current)

Setting iteration to iter 299.
6 is the walker with the maximum weight!

Indexing according to a seg_id returns all properties on the segment.
Children and parents have the same properties/keys.
['auxdata', 'bins', 'parents', 'pcoord', 'plotter', 'seg_id', 'states', 'weights']

Where did it come from?  Does it have any descendents?
23 is the parent!
[4 5 6] is/are the children!

Can we index either way?
[ True]

What properties exist in the current dictionary?
['aggregate_walkers', 'auxdata', 'bins', 'color_prob_evolution', 'conditional_flux_evolution', 'direct', 'instant_matrix', 'iteration', 'matrix', 'maxweight', 'minweight', 'parent', 'parents', 'pcoord', 'rate_evolution', 'raw', 'reweight', 'seg_id', 'state_pop_evolution', 'states', 'successful_trajectories', 'summary', 'target_flux_evolution', 'total_fluxes', 'walkers', 'weights']


In [3]:
# There's a built in plotting interface that can be used to plot interesting things!
# Try running this command directly from the terminal in w_ipa.
print(w.state_labels)
#w.current.direct.rate_evolution.plot(interface='matplotlib')
#print("Rate evolution from state 0 to 1")
w.current.direct.rate_evolution.plot(0,1,interface='matplotlib')
plt.xlabel("Iterations")
plt.ylabel("Flux")
print("Rate evolution from state 0 to 1")

State labels and definitions!
0: b'bound'
1: b'unbound'
2: Unknown
None
Rate evolution from state 0 to 1


In [4]:
import matplotlib.pyplot as pyplot
pyplot.plot(w.direct['target_flux_evolution']['expected'][:,0], color='blue')
#pyplot.plot(w.direct['target_flux_evolution']['ci_lbound'][:,0], color='blue')
#pyplot.plot(w.direct['target_flux_evolution']['ci_ubound'][:,0], color='gray')
pyplot.xlabel('Iteration')
pyplot.ylabel(u'Mean Flux (1/\u03c4)')
#pyplot.show()
pyplot.savefig("Ala.jpg",bbox_inches="tight",dpi=100)

In [5]:
w.current.direct.rate_evolution

rate_evolution data:
b'bound' -> b'unbound': mean=1.174348383337093e-03 CI=(5.740672741415509e-04, 2.009933853966615e-03) * tau^-1
b'unbound' -> b'bound': mean=6.931971585359309e-02 CI=(9.608989542704480e-03, 1.607124866359787e-01) * tau^-1
To access data, index via the following names:
['ci_lbound', 'ci_ubound', 'corr_len', 'error', 'expected', 'iter_start', 'iter_stop', 'name', 'plot', 'raw', 'sterr']

In [6]:
#Calculation of flux
m = 1.174348383337093e-03
tau = 2*np.power(1/10,12)
fl = m/(tau*np.power(10,9)) #in nanoseconds
print(fl)
print(1/fl)

0.5871741916685461
1.7030721278098848


In [7]:
#Calculation of simulation time
from westpa.analysis import Run
run = Run.open("west.h5")
num_iter = run.num_iterations
resampling_time = 2 #in ps
st = np.zeros((num_iter,1))
i = 1
while(i<=num_iter):
    iter_current = run.iteration(i)
    num_walkers = iter_current.num_walkers
    st[i-1] = num_walkers*resampling_time
    i = i + 1
ST = sum(st)/1000
print('total simulation time = ',ST,"ns")
run.close()
run.h5file

total simulation time =  [37.88] ns


<Closed HDF5 file>